**Importing all required libraries**

In [20]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import kagglehub
import os
import zipfile




In [18]:
# Step 1: Set up Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [19]:
# Step 3: Download the dataset
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset


Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
 99% 2.68G/2.70G [00:45<00:00, 134MB/s]
100% 2.70G/2.70G [00:45<00:00, 63.6MB/s]


In [21]:
zip_path = "new-plant-diseases-dataset.zip"
extract_path = "plant_dataset"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted!")

✅ Dataset extracted!


**Data Preprocessing**

Training image preprocessing

In [24]:
training_set = tf.keras.utils.image_dataset_from_directory(
    '/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
    # lables means the subdirectory names
    # which are used as labels for the images it finds automatically from the subdirectory names from the directory
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    verbose=True,
)

Found 70295 files belonging to 38 classes.


#### Validation image preprocessing


In [26]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    '/content/plant_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    verbose=True,
)

Found 17572 files belonging to 38 classes.


In [27]:
training_set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>

A Convolutional Neural Network (CNN) is a type of deep learning algorithm designed to automatically and adaptively learn patterns from image data, though it’s now also used for video, audio, and even text (in some NLP tasks).

Unlike standard neural networks (where every neuron is connected to every neuron in the next layer), CNNs use convolutional layers that apply filters (also called kernels) to input data, helping them detect spatial hierarchies — from edges to textures to object parts and whole objects.

1. **Input Layer**
You feed the network with a 2D image (for grayscale) or a 3D image (for RGB: height × width × channels).

2. **Convolution Layer**
Applies a small filter (e.g., 3×3 or 5×5) that slides over the image.

Produces a feature map showing where certain patterns (edges, corners) occur.

Multiple filters → multiple feature maps.

Think of this like applying an edge detection or blur filter in Photoshop.

3. **ReLU Activation (Rectified Linear Unit)
Adds non-linearity.**

Converts negative values in feature maps to zero: f(x) = max(0, x).

4. **Pooling Layer** (e.g., Max Pooling)
Reduces the size (downsampling) while keeping important features.

E.g., from 28×28 to 14×14 by selecting the max value in a 2×2 window.

5. **Stacking Multiple Conv + Pool Layers**
Deeper layers learn more complex patterns (e.g., face, object parts).

6. **Flattening**
Converts the final pooled feature map into a 1D vector.

7. **Fully Connected Layer**
Like traditional neural networks. Learns final classification boundaries.

Ends with a Softmax or Sigmoid layer for output (classification or regression).

**To Avoid Overshooting:**

1. choose a small learning rate. default is 0.001 we can use 0.0001
2. There may be chance of underfitting. So, we can increase the number of neurons.
3. Add more convolutional layers to extract more features from images there may be possibility to capture revelant features or models is confusing. due to lack of feature so feed with more features.

**Building the Model**

In [28]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Sequential

In [29]:
model = Sequential()

In [30]:
# Building Convulutional Layer

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

In [32]:
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

In [33]:
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

Now we have completed the convolutional layer and max pooling layer

No we need to flatten the output and add a fully connected layer and feed it to the nural network

In [34]:
model.add(Flatten())

In [35]:
model.add(Dense(units=1024, activation='relu'))

In [36]:
# Output Layer
model.add(Dense(units=38, activation='softmax'))

**Compaling the model**

In [37]:
model.compile(optimizer= tf.keras.optimizers.Adam(
    learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    16,778,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 38)             │        38,950 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,989,446 (68.62 MB)

 Trainable params: 17,989,446 (68.62 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.summary()

**Model Training**

In [38]:
training_history = model.fit(x=training_set, validation_data=validation_set, epochs=10)

Epoch 1/10
  14/2197 ━━━━━━━━━━━━━━━━━━━━ 2:37:22 4s/step - accuracy: 0.0377 - loss: 9.0183

KeyboardInterrupt: 